In [ ]:
import zipfile
import pandas as pd
import os

# Unzip the file
zip_path = "/mnt/data/Brazilian E-Commerce.zip"
extract_path = "/mnt/data/olist_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List files
files = os.listdir(extract_path)
files
